<a href="https://colab.research.google.com/github/dah4ik/GenAI-Project/blob/main/hybrid_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. SETUP AND LIBRARIES
# ==========================================================
!pip install -q ultralytics roboflow

import os
import cv2
import yaml
import shutil
from google.colab import drive
from ultralytics import YOLO
from roboflow import Roboflow

drive.mount('/content/drive')

# Paths based on your screenshots
BASE_DIR = '/content/drive/MyDrive/Shared drives/ChildSafetyProject/train/images'
SYNTHETIC_DIR = '/content/drive/MyDrive/Shared drives/genai_kids_room_project/synthetic_data'
RAW_IMAGES_DIR = os.path.join(BASE_DIR, 'raw_images')
FINAL_DATASET_DIR = os.path.join(BASE_DIR, 'final_combined_dataset')

# Create target directories
for split in ['train', 'val']:
    os.makedirs(os.path.join(FINAL_DATASET_DIR, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(FINAL_DATASET_DIR, split, 'labels'), exist_ok=True)

In [ ]:
# 2. MODELS INITIALIZATION (For Auto-Annotation)
# ==========================================================
RF_API_KEY = "Qq253w1BrhEg30BxzESZ"
rf = Roboflow(api_key=RF_API_KEY)

def get_latest_model(workspace_id, project_id):
    """Automatically fetches the latest version of the specified Roboflow project"""
    project = rf.workspace(workspace_id).project(project_id)
    latest_v = project.versions()[0].version_number
    print(f"Loaded: {project_id} v{latest_v}")
    return project.version(latest_v).model

# Base YOLOv8 model for standard objects (Knives=43, Scissors=76, Bottles=39)
base_model = YOLO('yolov8s.pt')

# Custom models from Roboflow Universe
battery_model = rf.workspace("battery-detector").project("battery-x0y91").version(2).model
lego_model    = rf.workspace("concours-objets-intelligents").project("lego-aprqg").version(2).model
coin_model    = rf.workspace("rohith-xznmb").project("coins-lknzw").version(1).model
power_model   = rf.workspace("cjbs").project("cattle-upi-power").version(2).model
plastic_model = rf.workspace("snowman1908").project("plastic-recyclable-detection").version(1).model

In [ ]:
# 3. MAPPING AND UTILS
# ==========================================================
MY_CLASSES = {
    'sharp_object': 0,
    'choking_hazard': 1,
    'electrical_hazard': 2,
    'chemical_danger': 3
}

def get_yolo_coords(pred, img_w, img_h):
    """Convert Roboflow dictionary prediction to YOLO string format"""
    xc = pred['x'] / img_w
    yc = pred['y'] / img_h
    nw = pred['width'] / img_w
    nh = pred['height'] / img_h
    return f"{xc:.6f} {yc:.6f} {nw:.6f} {nh:.6f}"

In [ ]:
# 4. STAGE 1: AUTO-ANNOTATE SYNTHETIC DATA
# ==========================================================
print("STAGE 1: Annotating synthetic data with Fallback...")

for class_name, class_id in MY_CLASSES.items():
    synth_path = os.path.join(SYNTHETIC_DIR, class_name, 'images')
    if not os.path.exists(synth_path): continue

    files = [f for f in os.listdir(synth_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    print(f"  Processing {class_name}...")

    for filename in files:
        img_path = os.path.join(synth_path, filename)
        image = cv2.imread(img_path)
        if image is None: continue
        h, w, _ = image.shape
        labels = []

        # AI Detection attempt
        if class_name == 'sharp_object':
            res = base_model(image)[0]
            for box in res.boxes:
                if int(box.cls) in [43, 76]:
                    xywh = box.xywhn[0].tolist()
                    labels.append(f"{class_id} {xywh[0]:.6f} {xywh[1]:.6f} {xywh[2]:.6f} {xywh[3]:.6f}")

        elif class_name == 'choking_hazard':
            for m in [battery_model, lego_model, coin_model]:
                preds = m.predict(img_path, confidence=30).json()['predictions']
                for p in preds: labels.append(f"{class_id} {get_yolo_coords(p, w, h)}")

        elif class_name == 'electrical_hazard':
            preds = power_model.predict(img_path, confidence=30).json()['predictions']
            for p in preds: labels.append(f"{class_id} {get_yolo_coords(p, w, h)}")

        elif class_name == 'chemical_danger':
            res = base_model(image)[0]
            for box in res.boxes:
                if int(box.cls) == 39:
                    xywh = box.xywhn[0].tolist()
                    labels.append(f"{class_id} {xywh[0]:.6f} {xywh[1]:.6f} {xywh[2]:.6f} {xywh[3]:.6f}")

        # --- FALLBACK: If AI found nothing in a dedicated folder, mark center ---
        if not labels:
            # 0.5 0.5 is center, 0.8 0.8 is 80% width/height
            labels.append(f"{class_id} 0.500000 0.500000 0.800000 0.800000")

        # Save to the FINAL dataset folder
        final_name = f"synth_{class_name}_{os.path.splitext(filename)[0]}"
        with open(os.path.join(FINAL_DATASET_DIR, 'train', 'labels', f"{final_name}.txt"), 'w') as f:
            f.write("\n".join(labels))
        cv2.imwrite(os.path.join(FINAL_DATASET_DIR, 'train', 'images', f"{final_name}.jpg"), image)

print("Stage 1 update complete.")

In [ ]:
# 5. STAGE 2: AUTO-ANNOTATE RAW PHOTOS
# ==========================================================
print("STAGE 2: Auto-annotating raw photos from RAW_IMAGES_DIR...")
raw_files = [f for f in os.listdir(RAW_IMAGES_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
image_counter = 0

for filename in raw_files:
    img_path = os.path.join(RAW_IMAGES_DIR, filename)
    image = cv2.imread(img_path)
    if image is None: continue
    h, w, _ = image.shape
    labels = []

    # YOLO Base (Knives/Scissors/Bottles)
    results = base_model(image)[0]
    for box in results.boxes:
        cid, xywh = int(box.cls), box.xywhn[0].tolist()
        coords = f"{xywh[0]:.6f} {xywh[1]:.6f} {xywh[2]:.6f} {xywh[3]:.6f}"
        if cid in [43, 76]: labels.append(f"{MY_CLASSES['sharp_object']} {coords}")
        elif cid == 39: labels.append(f"{MY_CLASSES['chemical_danger']} {coords}")

    # Roboflow Models (Batteries, Lego, Coins) -> Choking
    for m in [battery_model, lego_model, coin_model]:
        try:
            preds = m.predict(img_path, confidence=40).json()
            for p in preds['predictions']:
                labels.append(f"{MY_CLASSES['choking_hazard']} {get_yolo_coords(p, w, h)}")
        except Exception as e:
            print(f"      Warning: Failed to predict with {m} for {filename}: {e}")

    # Electrical Outlets -> Electrical Hazard
    try:
        e_preds = power_model.predict(img_path, confidence=40).json()
        for p in e_preds['predictions']:
            labels.append(f"{MY_CLASSES['electrical_hazard']} {get_yolo_coords(p, w, h)}")
    except Exception as e:
            print(f"      Warning: Failed to predict with power_model for {filename}: {e}")

    # Save results
    if labels: # Only save if detections were made
        with open(os.path.join(FINAL_DATASET_DIR, 'train', 'labels', filename.replace('.jpg', '.txt').replace('.png', '.txt')), 'w') as f:
            f.write("\n".join(labels))
        shutil.copy(img_path, os.path.join(FINAL_DATASET_DIR, 'train', 'images', filename))
        image_counter += 1
    else:
        print(f"      No detections made for raw image: {filename}. Skipping.")

print(f"STAGE 2 Complete. {image_counter} raw images annotated and added to dataset.")

In [ ]:
# 6. TRAINING
# ==========================================================
print("STAGE 3: Starting training...")
config = {
    'path': FINAL_DATASET_DIR,
    'train': 'train/images',
    'val': 'train/images',
    'names': {i: name for name, i in MY_CLASSES.items()}
}
with open(os.path.join(BASE_DIR, 'final_data.yaml'), 'w') as f:
    yaml.dump(config, f)

model = YOLO('yolov8n.pt')
model.train(data=os.path.join(BASE_DIR, 'final_data.yaml'), epochs=100, imgsz=640, name='safety_model_v3')

print("All stages complete. Model training finished.")

In [ ]:
import os
import glob
from IPython.display import Image, display


DRIVE_V33_PATH = '/content/drive/MyDrive/ChildSafetyProject/train_backups/safety_model_v33'

if os.path.exists(DRIVE_V33_PATH):
    print(f" Loading metrics from Google Drive: {DRIVE_V33_PATH}")


    metrics_to_show = [
        'confusion_matrix_normalized.png',
        'results.png',
        'F1_curve.png',
        'PR_curve.png'
    ]

    for metric in metrics_to_show:
        img_path = os.path.join(DRIVE_V33_PATH, metric)
        if os.path.exists(img_path):
            print(f"\n--- {metric} ---")
            display(Image(filename=img_path, width=800))
        else:
            # На некоторых твоих скриншотах файлы имеют префикс 'Box'
            alt_img_path = os.path.join(DRIVE_V33_PATH, f'Box{metric}')
            if os.path.exists(alt_img_path):
                display(Image(filename=alt_img_path, width=800))
            else:
                print(f"ℹ️ File {metric} not found in this folder.")
else:
    print(f" Error: Path not found. Please ensure Drive is mounted and folder name is correct.")